# 0. Play around google earth engine
Before we get started, you may play around google earth engine first. Note that it was "google earth engine" (GEE) rather than "google earth". Also, the programming language in the online GEE is Java, but don't worry, we can make it into Python.

1. You can browse around different datasets on GEE website.
2. check out the documentation on the GEE website. The codes and examples you may need are here.
3. Take a look at code editor and the example codes. Try to have an idea on the analysis procedure.

# 1. Authenticate and initialize Earth Engine

In [1]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00


In [2]:
!pip install earthengine-api --upgrade

In [3]:
!pip install geopandas

In [ ]:
import geemap # A Python package for interactive mapping with Google Earth Engine, ipyleaflet, and ipywidgets.
import ee

In [6]:
ee.Authenticate()

In [7]:
ee.Initialize(project = 'ee-luckyma1990')

# 2. Extract the image and plot

## America Example

In [16]:
# Load the Landsat 8 image
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

In [17]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.06,
 'CLOUD_COVER_LAND': 0.1,
 'COLLECTION_CATEGORY': 'T1',
 'COLLECTION_NUMBER': 2,
 'DATA_SOURCE_ELEVATION': 'GLS2000',
 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION': 'TIRS',
 'DATE_ACQUIRED': '2014-03-18',
 'DATE_PRODUCT_GENERATED': 1599866504000,
 'DATUM': 'WGS84',
 'EARTH_SUN_DISTANCE': 0.9953709,
 'ELLIPSOID': 'WGS84',
 'GEOMETRIC_RMSE_MODEL': 5.02,
 'GEOMETRIC_RMSE_MODEL_X': 3.38,
 'GEOMETRIC_RMSE_MODEL_Y': 3.711,
 'GEOMETRIC_RMSE_VERIFY': 2.215,
 'GRID_CELL_SIZE_PANCHROMATIC': 15,
 'GRID_CELL_SIZE_REFLECTIVE': 30,
 'GRID_CELL_SIZE_THERMAL': 30,
 'GROUND_CONTROL_POINTS_MODEL': 1078,
 'GROUND_CONTROL_POINTS_VERIFY': 307,
 'GROUND_CONTROL_POINTS_VERSION': 5,
 'IMAGE_DATE': '2014-03-18',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'K1_CONSTANT_BAND_10': 774.8853,
 'K1_CONSTANT_BAND_11': 480.8883,
 'K2_CONSTANT_BAND_10': 1321.0789,
 'K2_CONSTANT_BAND_11': 1201.1442,
 'LANDSAT_PRODUCT_ID': 'LC08_L1TP_044034_20140318_20200911_02_T1',
 'LANDSAT_SCENE_ID': 'LC8044

In [18]:
# Extract the red and near-infrared bands
red = image.select('B4')
nir = image.select('B5')

# Calculate NDVI
ndvi = nir.subtract(red).divide(nir.add(red))

# Create a visualization layer for the NDVI image
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401', '056201', '004C00', '023B01', '012E01', '011301']}

# Add the NDVI image to a map
Map = geemap.Map()
Map.addLayer(ndvi, ndvi_vis, 'NDVI')
Map.setCenter(-122.1899, 37.5010, 10) # Set the center of the map to San Francisco
Map

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## How to find a proper LandSat Image:


1. USGS EarthExplorer:

* Go to https://earthexplorer.usgs.gov/
* Select "Search Criteria", set up your interested time range and cloud coverage, and then search your interested place (e.g., Hong Kong).
* Click on "Data Sets" and select the Landsat Collection and Tier you are interested in (e.g., Landsat 8 Collection 2 Level 1)
* Click on "Results" and find the image that meets your criteria
* The Landsat image ID is the string of numbers and letters in the "Entity ID" column of the results table (e.g., for Landsat 8, the ID format is "LC08_L1TP_044034_20140318_20170421_01_T1")

2. Google Earth Engine Code Editor:

* Go to https://developers.google.com/earth-engine/datasets/
* Select your interested dataset (e.g., LandSat8)
* Search and obtain your image ID using the following codes

In [19]:
# Define the area of interest (AOI)
hkCenter = [114.19, 22.33]
aoi = ee.Geometry.Point(hkCenter)  # HK

# Define the date range
start_date = '2014-01-01'
end_date = '2023-04-01'

# Create a Landsat 8 image collection filtered by the AOI and date range
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
  .filterBounds(aoi) \
  .filterDate(start_date, end_date) \
  .sort('CLOUD_COVER').first()

# Get the Landsat 8 image ID for the first image in the collection
image_id = l8.id().getInfo()

# Print the image ID to the console
print('Landsat 8 image ID:', image_id)

Landsat 8 image ID: LC08_122044_20210220


## Let's try Hong Kong

In [20]:
# Load the Landsat 8 image
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_122044_20210220')

# Extract the red and near-infrared bands
red = image.select('B4')
nir = image.select('B5')

# Calculate NDVI
ndvi = nir.subtract(red).divide(nir.add(red))

# Create a visualization layer for the NDVI image
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401', '056201', '004C00', '023B01', '012E01', '011301']}

# Add the NDVI image to a map
Map1 = geemap.Map()
Map1.addLayer(ndvi, ndvi_vis, 'NDVI')
Map1.setCenter(hkCenter[0], hkCenter[1], 10)
Map1

Map(center=[22.33, 114.19], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [21]:
import json
import geopandas as gpd

In [23]:
gdf = gpd.read_file("/content/DC_16BC.shp")
Map1.add_gdf(gdf, layer_name="HK")
Map1


Map(bottom=114775.0, center=[22.2141061322513, 114.5482635498047], controls=(WidgetControl(options=['position'…

In [24]:
# Clip the NDVI image to the AOI
fc = geemap.gdf_to_ee(gdf)
ndvi_clipped = ndvi.clip(fc)
ndvi_clipped
Map1.addLayer(ndvi_clipped, ndvi_vis, 'NDVI')
Map1

Map(bottom=114666.0, center=[22.352524287064078, 114.15984870550443], controls=(WidgetControl(options=['positi…

## Yes, the image is not complete. We have to combine mutiple Landsat images and maybe better to use a HK shapefile

In [25]:
import json
import geopandas as gpd

In [26]:
hk_shape = gpd.read_file("/content/DC_16BC.shp")
geometry = geemap.gdf_to_ee(hk_shape)

In [28]:
# Define the date range
start_date = '2014-01-01'
end_date = '2023-04-01'

# Create a Landsat 8 image collection filtered by the AOI and date range
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
  .filterBounds(geometry) \
  .filterDate(start_date, end_date) \
  # .sort('CLOUD_COVER').first()

landsat_collection

In [29]:
landsat_image = landsat_collection.mosaic()
# mosaic composites images according to their position in the collection

# Extract the red and near-infrared bands
red = landsat_image.select('B4')
nir = landsat_image.select('B5')

# Calculate NDVI
ndvi = nir.subtract(red).divide(nir.add(red))

# Clip the NDVI image to the AOI
ndvi_clipped = ndvi.clip(geometry)

# Create a visualization layer for the NDVI image
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401', '056201', '004C00', '023B01', '012E01', '011301']}

In [30]:
# Add the NDVI image to a map
Map2 = geemap.Map()
Map2.addLayer(ndvi_clipped, ndvi_vis, 'NDVI')
# visparam = {"fill_color":"none", "outline":1, "outline_color":"red"}
# Map.addLayer(geometry,visparam,'HK')
Map2.centerObject(geometry, 10) # Set the center of the map to the AOI
Map2

Map(center=[22.3356701511706, 114.18715127899802], controls=(WidgetControl(options=['position', 'transparent_b…